In [3]:
! pip install datasets transformers sacrebleu -qq

In [4]:
ds_path='../../../data/dataset/lc-quad-wikidata-2021-07-11'

In [5]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm

In [6]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [7]:
raw_datasets["train"][0]

{'NNQT_question': 'What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}',
 'paraphrased_question': "What is Delta Air Line's periodical literature mouthpiece?",
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'sparql_dbpedia18': 'select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q188920> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P2813> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?obj . ?obj <http://www.wikidata.org/entity/P31> <http://wikidata.dbpedia.org/resource/Q1002697> } ',
 'sparql_wikidata': ' select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 } ',
 'subgraph': 'simple question right',
 'template': ' <S P ?O ; ?O instanceOf Type>',
 'template_index': 65,
 'uid': 19719}

In [8]:
rep_dict = {"ASK": "ask", "WHERE": "where", "SELECT": "select", "{": "[","}": "]", }
rep_dict2 = {"{": "","}": "" }
def replace_all(text, dict):
    for i, j in dict.items(): text = text.replace(i, j)
    return text

In [9]:
# Trim + Replace LowerCase + Remove weirdly long Question + replaceQ/P

In [10]:
df_property = pd.read_csv('../../../data/kdwd/property.csv').set_index('property_id')
del df_property['en_description']

In [11]:
df_property.head()

,en_label
property_id,
6,head of government
10,video
14,traffic sign
15,route map
16,highway system


In [53]:
df_property.loc[int('6')]

en_label    head of government
Name: 6, dtype: object

In [13]:
df_item = pd.read_csv('../../../data/kdwd/item_no_des.csv').set_index('item_id')

In [15]:
del df_item['Unnamed: 0']
df_item.head()

,en_label
item_id,
1,Universe
2,Earth
3,life
4,death
5,human


In [17]:
import re

In [123]:
def rplc_props(qry):
    for m in re.finditer(":Q\d+", qry):
        x = m.group(0)[1:]
        newstring = df_item.loc[int(x[1:])]['en_label'].replace(" ", "_").lower()
        qry = qry.replace(x, newstring)
        
    for m in re.finditer(":P\d+", qry):
        x = m.group(0)[1:]
        newstring = df_property.loc[int(x[1:])]['en_label'].replace(" ", "_").lower()
        qry = qry.replace(x, newstring)
    return qry

In [124]:
rplc_props('select ?obj where { wd:Q124057 p:P1411 ?s . ?s ps:P1411 ?obj . ?s pq:P1686 wd:Q3915489 }')

'select ?obj where { wd:dolores_del_río p:nominated_for ?s . ?s ps:nominated_for ?obj . ?s pq:for_work wd:la_otra }'

In [125]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = replace_all(d['sparql_wikidata'], rep_dict).strip()
            qry = rplc_props(qry)
            if len(d['question'])<250: 
                df = df.append({col: {'en':d['question'], 'sparql': qry}}, ignore_index=True)
            if d['paraphrased_question']!=[] and len(d['paraphrased_question'])<250:
                df = df.append({col: {'en':d['paraphrased_question'], 'sparql': qry}}, ignore_index=True)
        except: pass
    return df

In [126]:
df_test = prepare(raw_datasets["test"])

100%|██████████| 4781/4781 [00:25<00:00, 189.96it/s]


In [131]:
df_test.loc[0]['translation']

{'en': 'Who is the  {country} for {head of state} of {Mahmoud Abbas}',
 'sparql': 'select distinct ?sbj where [ ?sbj wdt:head_of_state wd:mahmoud_abbas . ?sbj wdt:instance_of wd:country ]'}

In [132]:
df_train = prepare(raw_datasets["train"])

100%|██████████| 19293/19293 [01:46<00:00, 181.08it/s]


In [133]:
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [134]:
ds_train

Dataset({
    features: ['translation'],
    num_rows: 38175
})

In [135]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [136]:
mother_ds.save_to_disk(ds_path)